# Export flux variables during 4-7 January 2022
- This script is used to export daytime and nighttime flux variables during 4-7 July 2022.
- Simulations: GM_SLUCM, GM_CLMU, GM_CLMU_ALB, GM_CLMU_NOAH. 
    - GM_CLMU_ALB: increase albedo for pervious floor and impervious floor.
    - GM_CLMU_NOAH: match urban land cover and parameter from GM_SLUCM.

In [1]:
import xarray as xr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
home_path = '/gws/nopw/j04/duicv/yuansun/'

In [2]:
ds_surf = xr.open_dataset(f'{home_path}0_WRFvsWRF-CTSM/input/WRF-CTSM/surfdata_1.2x1.2_SSP5-8.5_2022_78pfts_c250607.nc')
ds_urban = ds_surf['PCT_URBAN'].sum(dim='numurbl')
flat_index = ds_urban.argmax().item()

# Convert flat index back to (lsmlat, lsmlon) index
max_lat_idx, max_lon_idx = np.unravel_index(flat_index, ds_urban.shape)

# Get coordinate values
max_lat = ds_urban['lsmlat'].values[max_lat_idx]
max_lon = ds_urban['lsmlon'].values[max_lon_idx]
print(f"Max value: {ds_urban.max().item()} at lat={max_lat}, lon={max_lon}")

Max value: 88.24227244932186 at lat=18, lon=30


In [3]:
var_list = ['SWDOWN', 'GLW', 'HFX', 'LH', 'ALBEDO', 'EMISS', 'GRDFLX', 'TSK', 'T2', 'U10', 'V10', 'Q2', 'PSFC']
day_time = '13'
night_time = '01'
center_i = 30
center_j = 18
model_list = ['wrf', 'wrf-ctsm', 'wrf-ctsm_road_albedo', 'wrf-ctsm_uniform_urban'] # 'wrf-ctsm_road_albedo_init',
location_list = ['sp']
time_list = []
winter_date = ['01-04', '01-05', '01-06']
summer_date = ['07-16', '07-17', '07-18', '07-19'] #
for model in model_list: 
    for season in ['winter', 'summer']:
        if season == 'winter':
            date_list = winter_date
            if model =='wrf':
                ds_wrf = xr.open_dataset(f'{home_path}0_WRFvsWRF-CTSM/archive/d04_{model}/wrfout_d01_2022-01-04_00:00:00.nc')
            elif model == 'wrf-ctsm':
                ds_wrf = xr.open_dataset(f'{home_path}0_WRFvsWRF-CTSM/archive/d04_{model}/wrfout_d01_2022-01-04_00:00:00.nc')
            elif model == 'wrf-ctsm_uniform_urban':
                ds_wrf = xr.open_dataset(f'{home_path}0_WRFvsWRF-CTSM/sensitivity_summer_winter/archive/uniform_urban_noheating/wrfout_d01_2021-12-25_00:00:00.nc')
            elif model == 'wrf-ctsm_road_albedo':    
                ds_wrf = xr.open_dataset(f'{home_path}0_WRFvsWRF-CTSM/sensitivity_summer_winter/archive/road_albedo/wrfout_d01_2021-12-25_00:00:00.nc')
            #elif model == 'wrf-ctsm_road_albedo_init':    
                #continue
        elif season == 'summer':
            date_list = summer_date
            if model =='wrf-ctsm':
                ds_wrf = xr.open_dataset(f'{home_path}0_WRFvsWRF-CTSM/archive/d04_{model}/wrfout_d01_2022-07-15_01:00:00.nc')  
            elif model == 'wrf-ctsm_road_albedo':    
                ds_wrf = xr.open_dataset(f'{home_path}0_WRFvsWRF-CTSM/sensitivity_summer_winter/archive/road_albedo_hw/wrfout_d01_2022-07-16_01:00:00.nc')
            #elif model == 'wrf-ctsm_road_albedo_init':
                #ds_wrf = xr.open_dataset(f'{home_path}0_WRFvsWRF-CTSM/sensitivity_summer_winter/archive/road_albedo_july/wrfout_d01_2022-07-07_00:00:00.nc')
            elif model == 'wrf-ctsm_uniform_urban':
                #ds_wrf = xr.open_dataset(f'{home_path}0_WRFvsWRF-CTSM/sensitivity_summer_winter/archive/uniform_urban_noac/wrfout_d01_2022-07-07_00:00:00.nc')
                ds_wrf = xr.open_dataset(f'{home_path}0_WRFvsWRF-CTSM/sensitivity_summer_winter/archive/uniform_urban_noac_restart/wrfout_d01_2022-07-11_00:00:00.nc')
        for date in date_list:
            if (model =='wrf') & (season == 'summer'):
                if date in ['07-16', '07-17']:
                    ds_wrf = xr.open_dataset(f'{home_path}0_WRFvsWRF-CTSM/archive/d04_{model}/wrfout_d01_2022-07-13_01:00:00.nc')
                else:    
                    ds_wrf = xr.open_dataset(f'{home_path}0_WRFvsWRF-CTSM/archive/d04_{model}/wrfout_d01_2022-07-18_01:00:00.nc')
            day_time = pd.to_datetime(f'2022-{date} 13:00:00')
            night_time = pd.to_datetime(f'2022-{date} 01:00:00')
            time = pd.to_datetime(ds_wrf['XTIME'].values)
            idx_day = np.where(time == day_time)[0].item()
            print(idx_day, time[idx_day])
            idx_night = np.where(time == night_time)[0].item()
            day_sp = []
            night_sp = []
            for var in var_list:
                mean_day_sp = ds_wrf[var][idx_day:idx_day+2, center_j, center_i].mean().item()
                day_sp.append(mean_day_sp)
                mean_night_sp = ds_wrf[var][idx_night:idx_night+2, center_j, center_i].mean().item()
                night_sp.append(mean_night_sp)
            df_rows = pd.DataFrame([day_sp, night_sp], columns=var_list)
            #df_rows['time'] = [day_time, night_time]
            df_rows['model'] = model
            df_rows['location'] = 'sp'
            df_rows['tag'] = ['13:30', '01:30']
            df_rows['season'] = season
            time_list.append(df_rows)
df_time = pd.concat(time_list, ignore_index=True) 
df_time.head()

13 2022-01-04 13:00:00
37 2022-01-05 13:00:00
61 2022-01-06 13:00:00
84 2022-07-16 13:00:00
108 2022-07-17 13:00:00
12 2022-07-18 13:00:00
36 2022-07-19 13:00:00
13 2022-01-04 13:00:00
37 2022-01-05 13:00:00
61 2022-01-06 13:00:00
36 2022-07-16 13:00:00
60 2022-07-17 13:00:00
84 2022-07-18 13:00:00
108 2022-07-19 13:00:00
253 2022-01-04 13:00:00
277 2022-01-05 13:00:00
301 2022-01-06 13:00:00
12 2022-07-16 13:00:00
36 2022-07-17 13:00:00
60 2022-07-18 13:00:00
84 2022-07-19 13:00:00
253 2022-01-04 13:00:00
277 2022-01-05 13:00:00
301 2022-01-06 13:00:00
133 2022-07-16 13:00:00
157 2022-07-17 13:00:00
181 2022-07-18 13:00:00
205 2022-07-19 13:00:00


,SWDOWN,GLW,HFX,LH,ALBEDO,EMISS,GRDFLX,TSK,T2,U10,V10,Q2,PSFC,model,location,tag,season
0,204.606842,229.264465,27.708527,5.270475,0.154772,0.98,-31.851059,277.963959,277.136841,4.650197,-4.997540,0.003084,99828.687500,wrf,sp,13:30,winter
1,0.000000,323.772614,2.582034,0.915766,0.018000,0.98,17.732769,277.378204,277.242432,-1.498566,-3.316412,0.004284,99495.078125,wrf,sp,01:30,winter
2,205.213959,233.386963,10.221661,4.010367,0.154772,0.98,-58.172516,277.670654,277.258148,3.396661,-3.340973,0.003345,101032.179688,wrf,sp,13:30,winter
3,0.000000,231.481934,-40.027809,-0.270368,0.018000,0.98,26.362497,271.554138,274.329529,4.210200,-0.963764,0.003433,100242.906250,wrf,sp,01:30,winter
4,162.172302,270.687439,5.394849,1.583898,0.154772,0.98,-68.584435,276.833588,276.659943,0.752977,6.853031,0.004309,100443.531250,wrf,sp,13:30,winter


In [4]:
def compute_rh2_from_wrf(Q2, T2 , PSFC):
    # Convert T2 from Kelvin to Celsius
    T2_C = T2 - 273.15
    # Saturation vapor pressure (hPa)
    es = 6.112 * np.exp((17.67 * T2_C) / (T2_C + 243.5))

    # Actual vapor pressure (hPa), using PSFC (Pa)
    e = (Q2 * PSFC) / (0.622 + Q2) / 100  # Convert from Pa to hPa

    # Relative Humidity (%)
    RH2 = (e / es) * 100
    RH2 = np.clip(RH2, 0, 100)  # Ensure values are within 0-100%
    return RH2

In [9]:
def rh2_from_q2_t2_psfc_standard(Q2, T2, PSFC):
    """
    RH2 calculation (standard formulation).
    Q2: Specific humidity (kg/kg)
    T2: 2-m air temperature (K)
    PSFC: Surface pressure (Pa)
    Returns RH2 in %.
    """
    # Mixing ratio
    w = Q2 / (1 - Q2)

    # Temperature in Celsius
    T_C = T2 - 273.15

    # Saturation vapor pressure (Pa) using Tetens formula
    es = 610.94 * np.exp(17.625 * T_C / (T_C + 243.04))

    # Vapor pressure (Pa)
    e = (w * PSFC) / (0.622 + w)

    # Relative humidity
    RH = (e / es) * 100
    return np.clip(RH, 0, 100)


def rh2_from_q2_t2_psfc_hein(Q2, T2, PSFC):
    """
    RH2 calculation (Hein Zelle's simplified formula, tuned for WRF hindcasts).
    Q2: Specific humidity (kg/kg)
    T2: 2-m air temperature (K)
    PSFC: Surface pressure (Pa)
    Returns RH2 in fraction (0–1).
    """
    pq0 = 379.90516
    a2 = 17.2693882
    a3 = 273.16
    a4 = 35.86

    rh = Q2 / ((pq0 / PSFC) * np.exp(a2 * (T2 - a3) / (T2 - a4)))
    return np.clip(rh, 0, 1) * 100  # Convert to % and clip

In [5]:
df_time_average = df_time.groupby(['tag', 'model', 'location', 'season']).mean().reset_index()  
df_time_average['SWUP'] = df_time_average['SWDOWN'] * df_time_average['ALBEDO']
# Stefan-Boltzmann constant (W/m²/K⁴)
stefan_boltzmann = 5.67e-8
df_time_average['LWUP'] = df_time_average['EMISS'] * stefan_boltzmann * np.power(df_time_average['TSK'],4)
df_time_average['LWDOWN'] = df_time_average['EMISS'] * df_time_average['GLW']
df_time_average['netLW'] = df_time_average['LWDOWN'] - df_time_average['LWUP']
df_time_average['NET'] = df_time_average['SWDOWN'] - df_time_average['SWUP'] + df_time_average['netLW']
df_time_average['AHF'] = 0
#df_time_average['GRD'] = df_time_average['NET'] + df_time_average['AHF'] - df_time_average['HFX'] - df_time_average['LH'] 
df_time_average['W10M'] = np.sqrt(df_time_average['U10']**2 + df_time_average['V10']**2)
df_time_average['RH2M'] = compute_rh2_from_wrf(df_time_average['Q2'], df_time_average['T2'], df_time_average['PSFC'])
df_time_average.head()

,tag,model,location,season,SWDOWN,GLW,HFX,LH,ALBEDO,EMISS,...,Q2,PSFC,SWUP,LWUP,LWDOWN,netLW,NET,AHF,W10M,RH2M
0,01:30,wrf,sp,summer,0.0,350.552872,2.946615,-0.038948,0.018,0.98,...,0.007190,101627.517578,0.0,415.419271,343.541821,-71.877450,-71.877450,0,0.645768,47.709363
1,01:30,wrf,sp,winter,0.0,259.196594,-16.751774,0.162213,0.018,0.98,...,0.003577,100386.843750,0.0,308.372868,254.012667,-54.360200,-54.360200,0,1.489435,87.392488
2,01:30,wrf-ctsm,sp,summer,0.0,347.329132,-5.905338,0.274503,0.300,1.00,...,0.007437,101632.197266,0.0,409.267839,347.329132,-61.938707,-61.938707,0,0.787207,52.112415
3,01:30,wrf-ctsm,sp,winter,0.0,259.871821,1.757423,8.011357,0.300,1.00,...,0.003653,100385.346354,0.0,317.648946,259.871821,-57.777125,-57.777125,0,0.637025,85.576016
4,01:30,wrf-ctsm_road_albedo,sp,summer,0.0,347.611404,-5.893152,0.275079,0.300,1.00,...,0.007438,101632.257812,0.0,409.170493,347.611404,-61.559088,-61.559088,0,0.788793,52.192223


In [6]:
# anthropogenic heat
var_list = ['URBAN_AC', 'WASTEHEAT']
day_time = '13'
night_time = '01'
model_list = ['wrf-ctsm', 'wrf-ctsm_road_albedo'] #, 'wrf-ctsm_road_albedo_init'
location_list = ['sp']
# Get the time index from ds_wrf (assumes both have the same time)
time_list = []
for model in model_list: 
    for season in ['winter', 'summer']:
        if season == 'winter':
            date_list = winter_date
            if model == 'wrf-ctsm':
                ds_wrf = xr.open_dataset(f'{home_path}0_WRFvsWRF-CTSM/archive/d04_{model}/lnd/ctsm_lilac0.clm2.h0.2022-01-04-03600.nc')
            elif model == 'wrf-ctsm_road_albedo':    
                ds_wrf = xr.open_dataset(f'{home_path}0_WRFvsWRF-CTSM/sensitivity_summer_winter/archive/road_albedo/lnd/ctsm_lilac_road_albedo.clm2.h0.2022-01-01-03600.nc')
            #elif model == 'wrf-ctsm_road_albedo_init':    
                #continue
        elif season == 'summer':
            date_list = summer_date
            if model == 'wrf-ctsm':
                ds_wrf = xr.open_dataset(f'{home_path}0_WRFvsWRF-CTSM/archive/d04_{model}/lnd/ctsm_lilac4.clm2.h0.2022-07-15-03600.nc')  
            elif model == 'wrf-ctsm_road_albedo':
                ds_wrf = xr.open_dataset(f'{home_path}0_WRFvsWRF-CTSM/sensitivity_summer_winter/archive/road_albedo_hw/lnd/ctsm_lilac_road_albedo_hw.clm2.h0.2022-07-16-03600.nc')   
            #elif model == 'wrf-ctsm_road_albedo_init':    
                #ds_wrf = xr.open_dataset(f'{home_path}0_WRFvsWRF-CTSM/sensitivity_summer_winter/archive/road_albedo_july/lnd/ctsm_lilac_road_albedo_july.clm2.h0.2022-07-07-03600.nc') 
        time_list_model = []    
        for date in date_list:
            for date_time in [day_time, night_time]:
                datetime = pd.to_datetime(f'2022-{date} {date_time}:00:00')
                datetime_np = np.datetime64(datetime)
                wrf_times = ds_wrf['time'].values.astype('datetime64[ns]')
                time_index = np.argmin(np.abs(wrf_times - datetime_np))
                time_idx = ds_wrf.isel(time=time_index)
                datetime2 = datetime + pd.Timedelta(hours=1)
                datetime_np2 = np.datetime64(datetime2)
                time_index2 = np.argmin(np.abs(wrf_times - datetime_np2))
                time_idx2 = ds_wrf.isel(time=time_index2)
                rows_sp = []
                rows_region = []
                for var in var_list:
                    mean_data_sp = (time_idx[var][center_j, center_i].item() + time_idx2[var][center_j, center_i].item()) / 2
                    rows_sp.append(mean_data_sp)
                df_rows = pd.DataFrame([rows_sp], columns=var_list)
                df_rows['location'] = location_list
                df_rows['tag'] = ['13:30'] if date_time == day_time else ['01:30']
                time_list_model.append(df_rows)
        df_time_lilac = pd.concat(time_list_model, ignore_index=True)        
        df_time_mean = df_time_lilac.groupby(['tag', 'location']).mean().reset_index()
        df_time_mean['model'] = model
        df_time_mean['season'] = season
        time_list.append(df_time_mean)
df_time_mean = pd.concat(time_list, ignore_index=True)
df_time_mean['AHF'] = df_time_mean['URBAN_AC'] + df_time_mean['WASTEHEAT']
df_time_mean

,tag,location,URBAN_AC,WASTEHEAT,model,season,AHF
0,01:30,sp,0.0,5.087344,wrf-ctsm,winter,5.087344
1,13:30,sp,0.0,5.131047,wrf-ctsm,winter,5.131047
2,01:30,sp,0.0,0.000000,wrf-ctsm,summer,0.000000
3,13:30,sp,0.0,0.000000,wrf-ctsm,summer,0.000000
4,01:30,sp,0.0,5.088002,wrf-ctsm_road_albedo,winter,5.088002
5,13:30,sp,0.0,5.124968,wrf-ctsm_road_albedo,winter,5.124968
6,01:30,sp,0.0,0.000000,wrf-ctsm_road_albedo,summer,0.000000
7,13:30,sp,0.0,0.000000,wrf-ctsm_road_albedo,summer,0.000000


In [4]:
ds_check2 = xr.open_dataset(f'{home_path}0_WRFvsWRF-CTSM/sensitivity_summer_winter/archive/road_albedo_hw/lnd/ctsm_lilac_road_albedo_hw.clm2.h0.2022-07-16-03600.nc')
ds_check2_ac = ds_check2['URBAN_AC']
print(ds_check2_ac.max())

<xarray.DataArray 'URBAN_AC' ()> Size: 4B
array(0., dtype=float32)


In [8]:
df_time_lilac = df_time_mean[['tag', 'location', 'season', 'AHF', 'model']].copy()
df_time_lilac

,tag,location,season,AHF,model
0,01:30,sp,winter,5.087344,wrf-ctsm
1,13:30,sp,winter,5.131047,wrf-ctsm
2,01:30,sp,summer,0.000000,wrf-ctsm
3,13:30,sp,summer,0.000000,wrf-ctsm
4,01:30,sp,winter,5.088002,wrf-ctsm_road_albedo
5,13:30,sp,winter,5.124968,wrf-ctsm_road_albedo
6,01:30,sp,summer,0.000000,wrf-ctsm_road_albedo
7,13:30,sp,summer,0.000000,wrf-ctsm_road_albedo


In [9]:
df_time_average_modified = df_time_average.copy()
key = ['season','model','location','tag']
mapper = df_time_lilac.set_index(key)['AHF']

# Replace column
df_time_average_modified['AHF'] = [
    mapper.get(idx, 0) for idx in df_time_average_modified.set_index(key).index
]
df_time_average_modified['GRD'] = df_time_average_modified['NET'] + df_time_average_modified['AHF'] - df_time_average_modified['HFX'] - df_time_average_modified['LH'] 
df_time_average_modified.to_csv('data_for_figure/fluxes.csv', index=False)
df_time_average_modified

,tag,model,location,season,SWDOWN,GLW,HFX,LH,ALBEDO,EMISS,...,PSFC,SWUP,LWUP,LWDOWN,netLW,NET,AHF,W10M,RH2M,GRD
0,01:30,wrf,sp,summer,0.000000,350.552872,2.946615,-0.038948,0.018000,0.98,...,101627.517578,0.000000,415.419271,343.541821,-71.877450,-71.877450,0.000000,0.645768,47.709363,-74.785118
1,01:30,wrf,sp,winter,0.000000,259.196594,-16.751774,0.162213,0.018000,0.98,...,100386.843750,0.000000,308.372868,254.012667,-54.360200,-54.360200,0.000000,1.489435,87.392488,-37.770640
2,01:30,wrf-ctsm,sp,summer,0.000000,347.329132,-5.905338,0.274503,0.300000,1.00,...,101632.197266,0.000000,409.267839,347.329132,-61.938707,-61.938707,0.000000,0.787207,52.112415,-56.307873
3,01:30,wrf-ctsm,sp,winter,0.000000,259.871821,1.757423,8.011357,0.300000,1.00,...,100385.346354,0.000000,317.648946,259.871821,-57.777125,-57.777125,5.087344,0.637025,85.576016,-62.458562
4,01:30,wrf-ctsm_road_albedo,sp,summer,0.000000,347.611404,-5.893152,0.275079,0.300000,1.00,...,101632.257812,0.000000,409.170493,347.611404,-61.559088,-61.559088,0.000000,0.788793,52.192223,-55.941015
5,01:30,wrf-ctsm_road_albedo,sp,winter,0.000000,259.725667,1.653929,7.993065,0.300000,1.00,...,100385.364583,0.000000,317.600760,259.725667,-57.875093,-57.875093,5.088002,0.638749,85.569523,-62.434086
6,01:30,wrf-ctsm_uniform_urban,sp,summer,0.000000,346.945801,1.525779,0.434242,0.300000,1.00,...,101632.505859,0.000000,413.200312,346.945801,-66.254511,-66.254511,0.000000,0.740698,49.906203,-68.214532
7,01:30,wrf-ctsm_uniform_urban,sp,winter,0.000000,257.953135,-15.398008,-1.721585,0.300000,1.00,...,100388.356771,0.000000,308.024963,257.953135,-50.071828,-50.071828,0.000000,1.050911,91.751025,-32.952234
8,13:30,wrf,sp,summer,759.691040,370.210983,279.180534,2.531059,0.154772,0.98,...,101321.359375,117.579169,533.919043,362.806771,-171.112272,470.999598,0.000000,3.768751,24.866259,189.288005
9,13:30,wrf,sp,winter,190.664368,244.446289,14.441679,3.621580,0.154772,0.98,...,100434.799479,29.509574,329.453963,239.557368,-89.896595,71.258198,0.000000,2.974778,71.320903,53.194939
